In [1]:
import urllib2
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt, fabs, exp
from sklearn import linear_model
from sklearn.linear_model import enet_path
from sklearn.metrics import roc_auc_score, roc_curve

In [2]:
target_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data"
data = urllib2.urlopen(target_url)

xList  = []
labels = []

for line in data:
    row = line.strip().split(",")
    xList.append(row)
    
xNum = []

for row in xList:
    lastcol = row.pop()
    if lastcol == "M":
        labels.append(1.0)
    else:
        labels.append(0.0)
        
    attRow = [float(elt) for elt in row]
    xNum.append(attRow)
    
print (xList)

print ("bhavydfabdajfabdfhjakbhjlabfhak")
print (xNum)

# number of rows and columns
nrow = len(xNum)
ncol = len(xNum[0])

alpha = 1.0

# Calculate mean and variance
xMeans = []
xSD = []

for i in range(ncol):
    mean = sum([xNum[j][i] for j in range(nrow)])/nrow
    xMeans.append(mean)
    coldiff = [(xNum[j][i] - mean) for j in range(nrow)]
    sumSq = sum([coldiff[i] * coldiff[i] for i in range(nrow)])
    stdDev = sqrt(sumSq/nrow)
    xSD.append(stdDev)
    
# Normalize xNum
xNormalized = []
for i in range(nrow):
    rowNormalized = [(xNum[i][j] - xMeans[j])/xSD[j] for j in range(ncol)]
    xNormalized.append(rowNormalized)
    
# Normalize labels to center

meanLabel = sum(labels)/nrow
sdLabel = sqrt(sum([(labels[i] - meanLabel) * (labels[i] - meanLabel) for i in range(nrow)])/nrow)

labelNormalized = [(labels[i] - meanLabel)/sdLabel for i in range(nrow)]

Y = np.array(labelNormalized)
X = np.array(xNormalized)

alphas, coefs, _ = enet_path(xTrain, labelTrain,ll_ratio=0.8, fit_intercept=False, return_models=False)

plt.plot(alphas, coefs.T)
plt.xlabel("alpha")
plt.ylabel("Coefficients")
plt.axis("tight")
plt.semilogx()
ax = plt.gca()
ax.invert_xaxis()
plt.show()



URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [ ]:
nattr,nalpha = coefs.shape

# Find ceofficient ordering
nzList = []
for iAlpha in range(1, nalpha):
    coefList = list(coefs[:, iAlpha])
    nzCoef = [index for index in range(nattr) if coefList[index] != 0.0]
    for q in nzCoef:
        if not(q in nzList):
            nzList.append(q)
            
# Make up names for columns of X
names = ['V' + str(i) for i in range(ncol)]

nameList = [names[nzList[i]] for i in range(len(nzList))]
print ("Attrinutes ordered by how early they entered the model", nameList)


# Find coeff corrosponding to the best alpha value 
# Corrosponding to Normalized X and Y best alpha value is 0.020334883589342503

alphaStar = 0.020334883589342503
indexLTalphaStar = [index for index in range(100)if alphas[index] > alphaStar]
indexStar = max(indexLTalphaStar)

# Here the set of coefficients to deploy 
coefStar = list(coefs[:, indexStar])
print ("Best coefficient Values", coefStar)

# The coefficients on Normalized attributes gives another slightly different ordering

absCoef = [abs(a) for a in coefStar]

# sort by magnitude
coefSorted = sorted(absCoef, reverse=True)

idxCoefSize = [absCoef.index(a) for a in coefSorted if not(a == 0.0)]

nameList2 = [names[idxCoefSize[i]] for i in range(len(idxCoefSize))]

print ("Attributes ordered by Coef Size at Optimum Alpha", nameList2)